In [2]:
!pip install swig


[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
!pip install "gymnasium[box2d]"

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
   ---------------------------------------- 0.0/10.6 MB ? eta -:--:--
   -- ------------------------------------- 0.8/10.6 MB 4.2 MB/s eta 0:00:03
   ---- ----------------------------------- 1.3/10.6 MB 5.2 MB/s eta 0:00:02
   ---- ----------------------------------- 1.3/10.6 MB 5.2 MB/s eta 0:00:02
   ---- ----------------------------------- 1.3/10.6 MB 5.2 MB/s eta 0:00:02
   ----- ---------------------------------- 1.6/10.6 MB 1.5 MB/s eta 0:00:06
   ------ --------------------------------- 1.8/10.6 MB 1.4 MB/s eta 0:00:07
   -------- ------------------------------- 2.4/10.6 MB 1.6 MB/s eta 0:00:06
   ----------- ---------------------------- 3.1/10.6 MB 1.9 MB/s eta 0:00:04
   --------------- ------------------------ 4.2/10.6 MB 2.4 MB/s eta 0:00:03
   ------------------ --------------------- 5.0/10.6 MB 2.5 MB/s eta 0:00:03
   ---------------------- ----------------- 6.0/10.6 MB 2.


[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import numpy as np
import gymnasium as gym
import math
import torch.nn.functional as F

In [5]:
# Класс нейрона скрытого слоя 
# === Класс нейрона скрытого слоя ===
class Neuron:
    def __init__(self, input_size):
        self.weights = np.random.randn(input_size) * math.sqrt(2. / input_size)
        self.bias = 0.0

    def activate(self, x):
        z = np.dot(x, self.weights) + self.bias
        return np.tanh(z)
n = Neuron(input_size=8)
print(n.weights)
x = np.random.randn(8)
output = n.activate(x)
print("Neuron output:", output)

[-0.13703014  0.74272816  0.33811851  0.04608804  0.10973777  0.35198605
  0.02301245 -0.23384887]
Neuron output: 0.6080787177621686


In [14]:
# Популяция нейронов скрытого слоя 
class NeuronPopulation:
    def __init__(self, input_size, population_size):
        self.neurons = [Neuron(input_size) for _ in range(population_size)]

    def mutate(self, neuron, mutation_strength):
        new_neuron = Neuron(len(neuron.weights))
        new_neuron.weights = neuron.weights + np.random.normal(0, mutation_strength, size=neuron.weights.shape)
        new_neuron.bias = neuron.bias + np.random.normal(0, mutation_strength)
        return new_neuron

    def evolve(self, fitnesses, mutation_strength):
        sorted_indices = np.argsort(fitnesses)[::-1]
        top = [self.neurons[i] for i in sorted_indices[:len(self.neurons)//2]]
        new_population = []
        for neuron in top:
            new_population.append(neuron)
            new_population.append(self.mutate(neuron, mutation_strength))
        self.neurons = new_population[:len(self.neurons)]

pop = NeuronPopulation(input_size=8, population_size=5)
for i in range(len(pop.neurons)):
  print(pop.neurons[i])
print("Веса первого нейрона:", pop.neurons[0].weights)

mut_neuron = pop.mutate(pop.neurons[0], mutation_strength=0.1)
print("Мутировавшие веса:", mut_neuron.weights)

fitnesses = np.random.randn(5)
pop.evolve(fitnesses, mutation_strength=0.1)
print("Популяция эволюционировала - новые веса первого нейрона:", pop.neurons[0].weights)


Веса первого нейрона: [ 0.42567949  0.72282065 -0.82869737  0.71378388  0.25574556 -0.47463793
  0.04506419 -0.09617698]
Мутировавшие веса: [ 0.29202439  0.5614486  -0.64595706  0.56745452  0.10051797 -0.51312563
  0.01479519 -0.03920211]
Популяция эволюционировала - новые веса первого нейрона: [-0.41260744 -0.33737046  0.21474742  0.78204074  0.37456986  0.70607171
 -0.13434175 -0.86438971]


In [9]:
class OutputLayer:
    def __init__(self, hidden_size, output_size):
        self.weights = np.random.randn(hidden_size, output_size)

    def forward(self, hidden_output):
        return np.dot(hidden_output, self.weights)
    
ol = OutputLayer(hidden_size=4, output_size=3)
hidden_output = np.array([0.5, -0.2, 0.1, 0.9])
output = ol.forward(hidden_output)
print("Выход:", output)

Выход: [-1.35493425  0.0409108   0.01991938]


In [ ]:
class OutputPopulation:
    def __init__(self, hidden_size, output_size, population_size):
        self.layers = [OutputLayer(hidden_size, output_size) for _ in range(population_size)]

    def mutate(self, layer, mutation_strength):
        new_layer = OutputLayer(layer.weights.shape[0], layer.weights.shape[1])
        new_layer.weights = layer.weights + np.random.normal(0, mutation_strength, size=layer.weights.shape)
        return new_layer

    def evolve(self, fitnesses, mutation_strength):
        sorted_indices = np.argsort(fitnesses)[::-1]
        top = [self.layers[i] for i in sorted_indices[:len(self.layers)//2]]
        new_population = []
        for layer in top:
            new_population.append(layer)
            new_population.append(self.mutate(layer, mutation_strength))
        self.layers = new_population[:len(self.layers)]

pop = OutputPopulation(hidden_size=4, output_size=3, population_size=5)
for i in range(len(pop.layers)):
    print(pop.layers[i])

mut_layer = pop.mutate(pop.layers[0], mutation_strength=0.1)
print("Изначальные веса первого слоя:", pop.layers[0].weights)
print("Мутировавшие веса:", mut_layer.weights)

# случайные фитнесы
fitnesses = np.random.randn(5)
pop.evolve(fitnesses, mutation_strength=0.1)
print("Популяция эволюционировала - новые веса первого слоя:", pop.layers[0].weights)

Изначальные веса первого слоя: [[-0.30975242 -0.84171311  1.12921823]
 [-1.01035101 -0.34536254  0.78064809]
 [-1.36559108 -1.6858451   1.26461658]
 [ 0.20121101  1.01532726 -0.09413126]]
Мутировавшие веса: [[-0.36182279 -0.78561498  1.03448899]
 [-1.00042242 -0.33659015  0.60662937]
 [-1.38722548 -1.58220321  1.21170848]
 [ 0.28558941  0.89099227 -0.19327814]]
Популяция эволюционировала - новые веса первого слоя: [[-1.55439584  1.06818663 -0.34553246]
 [-0.31226088 -0.48521491  2.30782055]
 [ 1.80016657  0.07111576 -0.70082325]
 [-1.27062081 -0.65996415 -1.24357845]]


In [ ]:
# Полная сеть из скрытых нейронов и выходного слоя 
class Network:
    def __init__(self, neurons, output_layer):
        self.neurons = neurons
        self.output_layer = output_layer

    def forward(self, x):
        hidden_outputs = np.array([neuron.activate(x) for neuron in self.neurons])
        logits = self.output_layer.forward(hidden_outputs)
        return np.argmax(logits)